# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [1]:
# TODO: import needed libraries
import numpy as np
import nltk
import pandas as pd

Load the data in the file `random_headlines.csv`

In [3]:
# TODO: load the dataset
df = pd.read_csv("random_headlines.csv")
print(df.shape)
df.head()

(20000, 2)


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [4]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [7]:
# TODO: Preprocess the input data
df['tokens'] = df['headline_text'].apply(lambda row: nltk.word_tokenize(row))

df['alphanumeric'] = df['tokens'].apply(lambda row: [word for word in row if word.isalpha()])

stop = nltk.corpus.stopwords.words('english')
df['stop'] = df['alphanumeric'].apply(lambda row: [word for word in row if word not in stop])

stemmer = nltk.PorterStemmer()
df['stemmed'] = df['stop'].apply(lambda row: [stemmer.stem(word) for word in row])

df['stemmed'].head()

0    [ute, driver, hurt, intersect, crash]
1                       [die, cycl, accid]
2          [bumper, oliv, harvest, expect]
3    [replica, replac, northernmost, sign]
4          [wood, target, perfect, season]
Name: stemmed, dtype: object

In [9]:
!pip install --upgrade gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.23
    Uninstalling Cython-0.29.23:
      Successfully uninstalled Cython-0.29.23


Now use Gensim to compute a BOW

In [10]:
# TODO: Compute the BOW using Gensim
from gensim.corpora import Dictionary

dictionary = Dictionary(df['stemmed'])
corpus = [dictionary.doc2bow(line) for line in df['stemmed']]
print(np.shape(corpus))
corpus[0:2]

(20000,)


C:\Users\Luke Kelly\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1)]]

Compute the TF-IDF using Gensim

In [11]:
# TODO: Compute TF-IDF
from gensim.models import TfidfModel

tfidf_model = TfidfModel(corpus)
tf_idf = tfidf_model[corpus]
print(np.shape(tf_idf))

(20000,)


C:\Users\Luke Kelly\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [12]:
# TODO: Compute LSA
from gensim.models import LsiModel

lsi = LsiModel(corpus = corpus, num_topics = 4, id2word = dictionary)

For each of the topic, show the most significant words.

In [14]:
# TODO: Print the 3 or 4 most significant words of each topic
lsi.print_topics(num_words = 3)

[(0, '0.752*"polic" + 0.404*"man" + 0.208*"charg"'),
 (1, '0.670*"man" + -0.574*"polic" + 0.329*"charg"'),
 (2, '0.657*"new" + 0.296*"plan" + 0.241*"say"'),
 (3, '-0.701*"new" + 0.343*"say" + 0.336*"plan"')]

What do you think about those results?

Now let's try to use LDA instead of LSA using Gensim

In [17]:
# TODO: Compute LDA
from gensim.models import LdaModel

lda = LdaModel(corpus = corpus, num_topics = 4, id2word = dictionary, random_state = 0, chunksize = 512)

In [18]:
# TODO: print the most frequent words of each topic
lda.print_topics(num_words = 3)

[(0, '0.016*"report" + 0.011*"call" + 0.009*"may"'),
 (1, '0.012*"mine" + 0.010*"polic" + 0.008*"elect"'),
 (2, '0.011*"question" + 0.010*"council" + 0.010*"fund"'),
 (3, '0.013*"sydney" + 0.012*"charg" + 0.009*"interview"')]

Now, how does it work with LDA?

Let's make some visualization of the LDA results using pyLDAvis.

In [19]:
!pip install pyldavis

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136900 sha256=caf9efffa68c04ac9837d4b881d53768d3158e8c11291ed7baf5d9744d2acb69
  Stored in directory: c:\users\luke kelly\appdata\local\pip\cache\wheels\90\61\ec\9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=6eac37eb9131855f82724711378f7ca593382c19c0be7024eeab4b1eb29d089b
  Stored in directory: c:\users\luke kelly\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Su

In [25]:
# TODO: show visualization results of the LDA
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

C:\Users\Luke Kelly\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: No module named 'pyLDAvis.gensim'

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.